In [18]:
from sentence_transformers import SentenceTransformer
import torch
import spacy
import pandas as pd
import numpy as np
import concurrent.futures
import pickle
import faiss

nlp = spacy.load("es_core_news_sm")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 2070 SUPER


In [13]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [8]:
def divide_document(text, max_words=300):
    doc = nlp(text)
    sents = [sent.text for sent in doc.sents]
    fragments = []
    temp = ""

    for sent in sents:
        if len(temp.split()) + len(sent.split()) <= max_words:
            temp += " " + sent
        else:
            fragments.append(temp.strip())
            temp = sent

    if temp:
        fragments.append(temp.strip())

    return fragments

In [9]:
df = pd.read_csv("preprocessed.csv")
df

,Path,Text
0,pack\AIBR\2006-Aymaras-peruanos-y-chilenos-en-...,catalogada y caricaturizada bajo el rótulo de ...
1,pack\AIBR\2006-Cambios-de-género-y-discriminac...,agradecimientos la autora desea agradecer a la...
2,pack\AIBR\2006-Consecuencias-personales-en-la-...,has carried out the biggest employment adjustm...
3,pack\AIBR\2006-Construcción-de-modelos-de-géne...,tampoco queremos que se entienda identidad de ...
4,pack\AIBR\2006-Contribuciones-feministas-a-pro...,is irrelevant for others con el que teresa del...
...,...,...
1920,pack\Revista-Española-de-Antropología-American...,"cómo citar: burguete cal y mayor, araceli. 202..."
1921,pack\Revista-Española-de-Antropología-American...,project exposed some critical questionings. in...
1922,pack\Revista-Española-de-Antropología-American...,"1. introducción el interés por los quipus, que..."
1923,pack\Revista-Española-de-Antropología-American...,1. introducción ¿cómo se produce la diferencia...


In [11]:
def divide_and_assign_ids(i, text):
    if type(text) != str:
        return [], []

    frags = divide_document(text)
    return frags, [i] * len(frags)

fragments = []
fragments_ids = []

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(lambda row: divide_and_assign_ids(row[0], row[1]["Text"]),
                                df.iterrows()))

    for frags, ids in results:
        if not frags or not ids:
            continue

        fragments.extend(frags)
        fragments_ids.extend(ids)

len(fragments), len(fragments_ids)

(57716, 57716)

In [14]:
with open("fragments.pkl", "wb") as f:
    pickle.dump((fragments, fragments_ids), f)

In [16]:
embeddings = model.encode(fragments, convert_to_tensor=True, device=device)
print(embeddings.shape)

torch.Size([57716, 384])


In [17]:
embeddings_np = embeddings.cpu().numpy()
embeddings_np.shape

(57716, 384)

In [19]:
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000023FFF5C7D20> >

In [21]:
faiss.write_index(index, "index-sentence-transformers.faiss")

In [51]:
def search(query, top_k=10):
    query_embedding = model.encode([query.lower()], convert_to_tensor=True, device=device).cpu().numpy()
    _, indices = index.search(query_embedding, k=top_k*4)

    already_in = set()
    results = []
    for i in indices[0]:
        id = fragments_ids[i]
        if id in already_in:
            continue

        already_in.add(id)
        results.append(id)
        if len(results) == top_k:
            break

    return df.iloc[results]

In [53]:
q = """
La Guerra Hispano-Norteamericana, que tuvo lugar en 1898, fue un conflicto breve pero significativo que marcó el fin del Imperio español y el surgimiento de Estados Unidos como potencia global. En Filipinas, este conflicto tuvo un impacto profundo, ya que el archipiélago era una de las últimas colonias importantes de España en Asia.

El 1 de mayo de 1898, la flota estadounidense, al mando del comodoro George Dewey, derrotó a la armada española en la Batalla de la Bahía de Manila. Esta victoria fue decisiva y relativamente rápida, ya que la flota española, aunque valiente, estaba obsoleta y mal equipada en comparación con los buques modernos de Estados Unidos. La batalla se libró en cuestión de horas, y los estadounidenses lograron destruir o capturar todos los barcos españoles sin sufrir bajas significativas.

Tras la derrota española en la bahía de Manila, Estados Unidos tomó el control de la capital filipina. Sin embargo, este cambio de poder no significó la independencia inmediata para Filipinas. En lugar de ello, el archipiélago se convirtió en un nuevo territorio bajo control estadounidense, lo que llevó a la Guerra Filipino-Estadounidense (1899-1902), un conflicto sangriento entre las fuerzas estadounidenses y los independentistas filipinos que luchaban por su libertad.

La Guerra Hispano-Norteamericana en Filipinas no solo puso fin a más de tres siglos de dominio español en el archipiélago, sino que también marcó el inicio de una nueva era de influencia estadounidense en la región, con consecuencias duraderas para la política, la cultura y la sociedad filipinas.
"""

search(q)

,Path,Text
857,pack\Anuario-de-Estudios-Americanos\2022-Naval...,navalismo y comunidad hispánica. introducción ...
578,pack\Anuario-de-Estudios-Americanos\2009-Cober...,la migración extranjera en el perú republicano...
1347,pack\Revista-de-Indias\2019-Colonialismo-depor...,las deportaciones. una política del capitán ge...
947,pack\Revista-de-Indias\2001-Transición-polític...,la situación económica de la isla no era nada ...
909,pack\Revista-de-Indias\2000-La-guerra-hispanon...,la evolución de los hechos y acontecimientos h...
786,pack\Anuario-de-Estudios-Americanos\2019-La-de...,en 1739 el débil equilibrio que hacía veintisé...
471,pack\Anuario-de-Estudios-Americanos\2003-La-Co...,bajo trata de rescatar el proceso que llevó a ...
354,pack\Anuario-de-Estudios-Americanos\1997-Ameri...,"en el litoral pacífico, las acciones bélicas c..."
753,pack\Anuario-de-Estudios-Americanos\2017-Regal...,introducción desde 1765 hasta 1784 un grupo de...
433,pack\Anuario-de-Estudios-Americanos\2001-El-ho...,italia. francia y españa coincidían en la defe...
